# Analyzing the Wine Dataset

#### In this analysis, you will use two machine learning methods implemented in python to predict the cultivar of wine based on its chemical features.

#### Please take your time with this analysis. Run each command (using Shift + Enter) and think about what you are asking the computer to do.

#### The wine dataset is the result of a chemical analysis of wines grown in the same region in Italy but derived from three different cultivars. The analysis determined the quantities of 13 constituents found in each of the three types of wines. Though initially used for fairly naive discrimination methods, it is an early example of machine learning for predictive data in biology.

#### The first thing we will do is load the data, which is included as part of the sklearn python module. If you end up using machine learning in your practcal projects, it is likely that sklearn will be of use to you.

In [ ]:
# import the function to load the dataset
from sklearn.datasets import load_wine

# load the dataset
wine_dataset = load_wine()

#### We can quickly check a few things to ensure our data are loaded.

> #### 1. The column headers (keys)
> #### 2. The values of the first 5 columns of data
> #### 3. The cultivar each row's data belong to (Targets)
> #### 4. The names of those cultivars
> #### 5. The feature (variable) names

In [ ]:
# print the keys
print("Keys of wine_dataset: \n{}".format(wine_dataset.keys()))
print("\n")

# print the first five columns
print("First five columns of data:\n{}".format(wine_dataset['data'][:5]))
print("\n")

# print the targets
print("Targets:\n{}".format(wine_dataset['target'][:]))
print("\n")

# print the target names
print("Target names:\n{}".format(wine_dataset['target_names']))
print("\n")

# print the feature names
print("Feature names:\n{}".format(wine_dataset['feature_names']))

#### We can also look at the description of the dataset provided by one of the hardworking folks at sklearn

In [ ]:
# print the description
print(wine_dataset['DESCR'])

#### OK - we can be fairly confident that our data are loaded correctly.

#### Now we are going to filter our data set to contain only 4 variables, to make it slightly easier to work with.

In [ ]:
# import the modules we need
import numpy as np

# choose the features we want to keep
selected_features = ['alcohol', 'malic_acid', 'total_phenols', 'flavanoids']

# find their column numbers
indices = [wine_dataset.feature_names.index(f) for f in selected_features]

# filter the data
filtered_data = wine_dataset.data[:, indices]

# update the format of the new dataset
from sklearn.utils import Bunch
wine = Bunch(
    data=filtered_data,
    target=wine_dataset.target,
    feature_names=selected_features,
    target_names=wine_dataset.target_names,
    DESCR=wine_dataset.DESCR,
    frame=None
)

# check the format of our filtered dataset looks right
print(wine.feature_names)
print(wine.data.shape)

#### Now we need to set our data up for analysis. We are using the handy python function "train_test_split" from the sklearn module. This converts our data into a test set and a training set. Have a look at how the funciton is implemented here and consider the arguments. random_state is set to 0 - don't worry about this. It's just the random seed and by fixing it we guarentee that we all get the same result. Normally, this would not be set.


### Questions
>#### 1. Why do we split our dataset into a training set and a test set?
>#### 2. What proportion of the dataset will be included in the training set?
>#### 3. Given our use of train_test_split, do you think the algorithms we will be using are supervised, or unsupervised method?



In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
        wine['data'], wine['target'], test_size=0.25, random_state=0)
print("X_train shape: {}".format(X_train.shape))
print("y_train shape: {}".format(y_train.shape))
print("X_test shape: {}".format(X_test.shape)) 
print("y_test shape: {}".format(y_test.shape))

#### The output of the above tells you the dimentions of the data - what it's saying is that, for example, the training dataset is 133 wines with 4 features plus the Y variable (cultivar)

#### Now we will convert the data into a ["pandas"](https://pandas.pydata.org/docs/user_guide/index.html) dataframe. As far as I'm aware, pandas has nothing to do with the [black and white balls of clumsyness that they have in Edinburgh zoo](https://www.wwf.org.uk/learn/fascinating-facts/pandas). More boringly, it is a set of functions that allow us to interpret python data in data-frames, like you might be used to in R. A lot of sklearn functions rely on pandas data structures.

#### We can then plot the 4 measurement variables against each other, observing how they correlate and how cultivars tend to cluster with each other. We call this a [scatter matrix](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.plotting.scatter_matrix.html).

In [ ]:
# import the modules we will use
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# convert the data to a pandas dataframe
wine_dataframe = pd.DataFrame(X_train, columns=wine.feature_names)

# plot the four measurements against each other
grr = pd.plotting.scatter_matrix(wine_dataframe, c=y_train, figsize=(15, 15), marker='o',hist_kwds={'bins': 20}, s=60, alpha=.8)
plt.figure()
plt.imshow([np.unique(wine.target)])
_ = plt.xticks(ticks=np.unique(wine.target),labels=wine.target_names)

### Questions
>#### 4. Given the scatter matrix above, which two features will be most likely to create an accurate model with which to predict the cultivar of wine?

>#### 5. Which two features would be the least likely to create an accurate model?

#### Right. Now you get to do some actual computation. We will be implementing the [K Neighbours](https://scikit-learn.org/stable/modules/neighbors.html#nearest-neighbors-classification) algorithm to create a model to predict the cultivar of wine from the amount of total phenols and flavenoids, which you hopefully identified as the most informative pair of features above. K Neighbours is a very simple method whose model is built simply by asking for a datapoint, the identify of the nearest K neighbours. The majority vote amounts to the prediction for that point. There is one parameter - n_neighbors (K). This is the size of the sample of nearest neighbours to make a prediction from. i.e. K=1 - the prediction is the identity of the nearest neighbour. K=3 - the prediction is the identity of the highest number of the nearest 3 neighbours. In our case, we will set this to one. Have a think about how changing this might affect your predictions.

#### Technically, unless we are estimating K, which we are not here, K nearest neighbours is not machine learning, but a lot of the principles that we learned in the lecture apply.

#### The way that sklearn works is, to my mind, a bit unintuitive. First, you create a variable that features all of the hyperparameters with which you can make your model (here that variable is knn).

In [ ]:
# import the function we will use
from sklearn.neighbors import KNeighborsClassifier

# create the model classifier variable
knn = KNeighborsClassifier(n_neighbors=1)

#### Then you can fit the model on the data - we have here chosen to fit it to the total phenols and flavenoids `(X_train[:,2:5])`

In [ ]:
# fit the model to our data
knn.fit(X_train[:,2:5], y_train)

#### We can visualise our model using the following code - don't worry too much about the code. Just know that it first sets up the plot area, then colours the plot according to the mode estimated, then plots the training data and in a colour corresponding to the cultivar according to a legend.

#### The code below produces a warning about colours which is to boring to concern us. Don't worry about it.

In [ ]:
# import the functions we will use
from matplotlib.colors import ListedColormap
import matplotlib.pyplot as plt

# plot the model and the data
n_neighbors = 1
x_min,x_max = X_train[:,2].min() - 1, X_train[:,2].max()+ 1
y_min,y_max = X_train[:,3].min() - 1, X_train[:,3].max()+ 1
h=0.02
xx,yy = np.meshgrid(np.arange(x_min,x_max,h),np.arange(y_min,y_max,h))
Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

cmap_bold = ListedColormap(['darkorange', 'c', 'darkblue'])
cmap_light=ListedColormap(['orange', 'cyan', 'cornflowerblue'])

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.pcolormesh(xx, yy, Z, cmap=cmap_light, shading='gouraud')
for target in wine_dataset.target_names:
    index=np.where(wine_dataset.target_names==target)[0][0]
    ax1.scatter(X_train[:,2][y_train==index],X_train[:,3][y_train==index],
                cmap=cmap_bold,edgecolor='k', s=20, label=target)
ax1.set_xlim(x_min,x_max)
ax1.set_ylim(y_min,y_max)
ax1.legend()
ax1.set_xlabel("total phenols")
ax1.set_ylabel("flavenoids")
ax1.set_title("3-Class classification (k = %i, weights = '%s')"
              % (n_neighbors, 'uniform'))
plt.show()

#### Have a look at your "model" and think about what this means for making predictions in the test set.

### Questions
>#### 6. What combinations of total phenols and flavenoids are likely to be difficult to predict in the test set based on the figure?
>#### 7. It is impossible to say for sure without extra analyses, but from the figure, do you think we've overfit or underfit the model to the data?

#### The next step is to predict the cultvar for a new, unknown wine. Imagine you found a new wine and took measurements of 4 for total phenols and 3.5 for flavenoids. What cultivar does our model propose this wine belongs to? Remember, you don't actually know for sure in this case, though it's pretty clear cut here.

In [ ]:
# generate a data entry for our new wine
new_data=np.array([[13,2.34,4,3.5]])

# predict which cultivar this wine is
prediction = knn.predict(new_data[:,2:5])

# print the prediction
print("Prediction: {}".format(prediction))
print("Predicted target name: {}".format(wine_dataset['target_names'][prediction]))

#### Now is the moment of truth. Does our model predict the cultivar well on an independent dataset (our test set)?

#### To ask this question, we throw the X values (total phenols and flavenoids) of the test set at our model and return the predictions. After this, you can print the true values of the cultivar (0 = class_0, 1 = class_1, 2 = class_2) and the predicted values.

In [ ]:
# predict the cultivars of our test dataset
y_pred = knn.predict(X_test[:,2:5])

# print the predictions and then the true values
print("\nTest set predictions:\n {}".format(y_pred))
print("\nTest set true values:\n {}".format(y_test))

# print whether or not the prediction matched the true value
print("\nPredictions matched true values:\n {}".format(y_pred == y_test))

### Questions
>#### 8. How many wines were asigned to the wrong cultivar?

#### We can also ask how accurate the model is - in this case simply as a measure of the proportion of corrections that were correct of the total number of predictions

In [ ]:
# print the proportion of predictions that were correct
print("Test set score: {:.2f}".format(knn.score(X_test[:,2:5], y_test)))

#### We can plot the values for the test set on the same axes that we plotted the training points.

#### As before, ignore the warning

In [ ]:
# plot the model again with the values from the test set as points
x_min,x_max = X_train[:,2].min() - 1, X_train[:,2].max()+ 1
y_min,y_max = X_train[:,3].min() - 1, X_train[:,3].max()+ 1
h=0.02
xx,yy = np.meshgrid(np.arange(x_min,x_max,h),np.arange(y_min,y_max,h))
Z = knn.predict(np.c_[xx.ravel(), yy.ravel()])
Z = Z.reshape(xx.shape)

cmap_bold = ListedColormap(['darkorange', 'c', 'darkblue'])
cmap_light=ListedColormap(['orange', 'cyan', 'cornflowerblue'])

fig = plt.figure()
ax1 = fig.add_subplot(111)
ax1.pcolormesh(xx, yy, Z, cmap=cmap_light, shading='gouraud')
for target in wine.target_names:
    index=np.where(wine.target_names==target)[0][0]
    ax1.scatter(X_test[:,2][y_test==index],X_test[:,3][y_test==index],
                cmap=cmap_bold,edgecolor='k', s=20, label=target)
ax1.set_xlim(x_min,x_max)
ax1.set_ylim(y_min,y_max)
ax1.legend()
ax1.set_xlabel("total phenols")
ax1.set_ylabel("flavenoids")
ax1.set_title("3-Class classification (k = %i, weights = '%s')"
              % (n_neighbors, 'uniform'))
plt.show()

#### Ask yourself how the model performs. Were you right about the range in which prediction would be more difficult?

### Questions
>#### 9. Can you think of any downsides of the K nearest neighbours algorithm for this dataset?
>#### 10. In a hypothetical scenario, you are given a load more data and your accuracy reduces to .5. Upon investigation you find that your model is overfit. This means that the model is inferring patterns that are true only of the training set, meaining they are not generalisable. How might you adapt your strategy to reduce overfitting but still using K nearest neighbours?

### Random forests

#### Assuming no dreadful miscalculation of timings on the part of your instructor, you will have heard about [Random Forests](https://towardsdatascience.com/random-forest-3a55c3aca46d) in the lecture part of this taster session. I hope you were paying attention.

#### Just in case you weren't, Random Forest is a machine learning algorithm that asks questions of our data using decision trees. The model is built by slowly splitting the dataset into purer classes (i.e. moving towards all cultivar 0 or cultivar 2 etc.) according to the X values. A tree might split the data into those with total phenols over 3 and those with flavenoids under 3. Those cultivars with malic acid over 2 might then be split into those with alcohol over 90. This happens sequentially until the nodes of the tree are at some desired purity level (in this case complete purity) or the tree reaches a maximum depth (in this case 8). The random forest generates a given number of trees to make its predictions, here 100.

#### Unlike K nearest neighbours above, we will use all 4 features to create our model here. In principle, Random Forests should rely on the most informative features without us specifying them so there isn't much need to worry about which features will be most informative, though too many completely useless features will reduce the efficacy of the model.

#### As we've seen, in sklearn, you set up the model first then fit it to your data, which is done below. Have a look at the parameters and think about how the analysis might change under different hyperparameters. To read what each of these parameters does, you can look at the [sklearn Random Forest classifier documentation](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestClassifier.html)

In [ ]:
# import the function we will use
from sklearn.ensemble import RandomForestClassifier

# set up the model
model = RandomForestClassifier(n_estimators = 100,
                                       max_depth = 8,
                                       max_features='sqrt',
                                       min_samples_split=2,
                                       n_jobs=1,
                                       random_state=0)

# fit the model to our training set
model.fit(X_train, y_train)

#### To give you an idea of what the model actually looks like, here is some code that plots the first of the trees in your forest, which should all look identical because you've been given the same starting seed. In each box you are given:
>#### a) The decision being made (how the dataset is being split). This is missing in terminal nodes.
>#### b) The [GINI importance](https://sam-black.medium.com/calculating-a-features-importance-with-xgboost-and-gini-impurity-3beb4e003b80) of that node. Don't worry too much about GINI unless you are especially interested. For now, you can think of it as the efficacy with which that node splits the tree into purer samples
>#### c) The number of samples (wines) that have taken the path leading to that node
>#### d) The value - number of samples of each cultivar that have taken the path leading to that node, formatted as [class_0, class_1, class_2]
>#### e) the class that would be predicted if a new wine took that path through the tree.

#### NOTE: when a wine satisfies the condition set out in the decision in a box it always passes to the left-hand box below and wines that do not satisfy the condition follow the branch on the right-hand side!

In [ ]:
# import the function we will use
from sklearn import tree

# set up the variables and figure attributes
fn=wine['feature_names']
cn=wine['target_names']
fig, axes = plt.subplots(nrows = 1,ncols = 1,figsize = (4,4), dpi=800)

# create the tree plot
tree.plot_tree(model.estimators_[0],
               feature_names = fn, 
               class_names=cn,
               filled = True);

### Questions
>#### 11. How many wines in the training set have flavanoids >= 2.31?
>#### 12. Which cultivar would a wine with flavenoids 2,  alcohol 12, malic acid 2 and total phenols 2.5 be predicted to be by the tree displayed here?
>#### 13. What is the route taken through this tree by the majority of wines?

#### Now we predict the cultivar for the wines in our test set and compare to the true values:

In [ ]:
# predict the cultivar of the wine's in our test set
y_pred_test = model.predict(X_test)

# print the test set predictions and true values
print("Test set predictions:\n {}".format(y_pred_test))
print("\nTest set true values:\n {}".format(y_test))

# print the proportion of test set predictions that were correct
print("\nTest set score: {:.2f}".format(model.score(X_test, y_test)))

#### So, our random forest has performed much better than our K nearest neighbours analysis. It has predicted the cultivar of far more of the wines in the test set correctly.

#### There is an added benefit of the Random Forest algorithm over K nearest neighbours. Remember when I said you don't need to worry too much about GINI? Well the commands below will print the values of GINI for each of the 4 different features. Again, we don't need to worry too much about the specifics of GINI, but you can know that it gives you an idea of the relative importance of each of the features in building the model.

In [ ]:
# print the gini values for each of the features in our dataset
for i in range(4):
    print(wine['feature_names'][i])
    print(model.feature_importances_[i].round(3))

### Questions
>#### 14. Order the variables in most important to least important. Compare with the scatter matrix. Does this fit your expectations?

#### That's the base practical done. Well done. If you are finished but your appetite for machine learning has not yet been satiated, you can play about with the code to your heart's content. Here are some suggestions.
>#### Change the parameters of the K nearest neigbours algorithm and rerun, comparing results with the basic ones.
>#### Change the parameters of the Random Forest part of the practical and see how results change. You might find it intersting to see how results change with only 1 tree of max depth 2 or something. This wouldn't be a good analysis but could show how much is acheived by just one feature
>#### Implement your own sklearn algorthm on the data - perhaps a neural network. This might be very difficult - I've not done it myself.

#### If you need any help with the controls of the jupyter notebook, let the instructor(s) know.